In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from pathlib import Path

import hydra
import torch
import wandb
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

from opr.datasets.dataloader_factory import make_dataloaders
from opr.testing import test
from opr.training import epoch_loop
from opr.utils import flatten_dict, set_seed

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
from hydra import initialize, compose
from hydra.utils import instantiate

with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name='config_nclt_text.yaml')  # only_text
    # cfg = compose(config_name='config.yaml')  # lidar + cam

In [4]:
cfg.dataset

{'dataset': {'_target_': 'opr.datasets.nclt.NCLTDataset', 'dataset_root': '/home/docker_opr/Datasets/NCLT_preprocessed', 'modalities': '${general.modalities}', 'images_subdir': 'lb3_small/Cam5', 'text_embs_dir': 'clip-vit-base-patch32', 'mink_quantization_size': 0.5, 'coords_limit': [-100, 100]}, 'sampler': {'_target_': 'opr.datasets.samplers.batch_sampler.BatchSampler', 'batch_size': 8, 'batch_size_limit': 160, 'batch_expansion_rate': 1.4, 'positives_per_group': 2, 'seed': '${general.seed}'}, 'num_workers': '${general.num_workers}'}

In [5]:
cfg.model

{'_target_': 'opr.models.base_models.ComposedModel', 'image_module': None, 'cloud_module': None, 'fusion_module': None, 'text_module': {'_target_': 'opr.models.base_models.MultiTextModule', 'text_module': {'_target_': 'opr.models.base_models.TextModule', 'text_emb_size': 512, 'hidden_size': 512}, 'fusion_module': {'_target_': 'opr.models.fusion.Concat'}}}

In [6]:
print("=> Instantiating model...")
model = instantiate(cfg.model)

print("=> Instantiating loss...")
loss_fn = instantiate(cfg.loss)

print("=> Making dataloaders...")
dataloaders = make_dataloaders(
    dataset_cfg=cfg.dataset.dataset,
    batch_sampler_cfg=cfg.dataset.sampler,
    num_workers=cfg.dataset.num_workers,
)

print("=> Instantiating optimizer...")
params_list = []
modalities = list(set([m.split("_")[0] for m in cfg.general.modalities]))
for modality in modalities:
    params_list.append(
        {
            "params": getattr(model, f"{modality}_module").parameters(),
            "lr": cfg.optimizer.learning_rates[f"{modality}_lr"],
        }
    )
optimizer = instantiate(cfg.optimizer.fn, params=params_list)
print("Instantiating scheduler...")
scheduler = instantiate(cfg.scheduler, optimizer=optimizer)

model = model.to(cfg.general.device)

=> Instantiating model...
=> Instantiating loss...
=> Making dataloaders...
=> Instantiating optimizer...
Instantiating scheduler...


In [7]:
# batch, _, _ = next(iter(dataloaders["train"]))
# batch["text_emb_cam1"]

In [8]:
# dataloaders["train"].dataset[0].keys()

In [9]:
model, optimizer, modalities

(ComposedModel(
   (text_module): MultiTextModule(
     (text_module): TextModule(
       (fc1): Linear(in_features=512, out_features=512, bias=True)
       (relu): ReLU()
       (fc2): Linear(in_features=512, out_features=512, bias=True)
     )
     (fusion_module): Concat()
   )
 ),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     initial_lr: 0.001
     lr: 0.001
     maximize: False
     weight_decay: 0.0001
 ),
 ['text'])

In [10]:
if not cfg.general.debug and not cfg.wandb.disabled:
    config_dict = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)
    wandb.init(
        name=cfg.wandb.run_name,
        project=cfg.wandb.project,
        settings=wandb.Settings(start_method="thread"),
        config=config_dict,
    )
    wandb.save(f"configs/{wandb.run.name}.yaml")
    run_name = wandb.run.name
else:
    run_name = "debug"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: petili. Use `wandb login --relogin` to force relogin


In [11]:
checkpoints_dir = (
    Path(cfg.general.checkpoints_dir) / f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}_{run_name}"
)
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True)

In [12]:
best_recall_at_1 = 0.0

# for epoch in range(60, 120):
for epoch in range(cfg.general.epochs):
    print(f"\n\n=====> Epoch {epoch+1}:")
    # TODO: resolve mypy typing here
    train_batch_size = dataloaders["train"].batch_sampler.batch_size  # type: ignore
    val_batch_size = dataloaders["val"].batch_sampler.batch_size  # type: ignore

    print("\n=> Training:\n")

    train_stats, train_rate_non_zero = epoch_loop(
        dataloader=dataloaders["train"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        phase="train",
        device=cfg.general.device,
    )

    print(f"\ntrain_rate_non_zero = {train_rate_non_zero}")

    batch_expansion_th = cfg.general.batch_expansion_th
    if batch_expansion_th is not None:
        if batch_expansion_th == 1.0:
            print("Batch expansion rate is set to every epoch. Increasing batch size.")
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore
        elif train_rate_non_zero is None:
            print(
                "\nWARNING: 'batch_expansion_th' was set, but 'train_rate_non_zero' is None. ",
                "The batch size was not expanded.",
            )
        elif train_rate_non_zero < batch_expansion_th:
            print(
                "Average non-zero triplet ratio is less than threshold: ",
                f"{train_rate_non_zero} < {batch_expansion_th}",
            )
            # TODO: resolve mypy typing here
            dataloaders["train"].batch_sampler.expand_batch()  # type: ignore

    print("\n=> Validating:\n")

    val_stats, val_rate_non_zero = epoch_loop(
        dataloader=dataloaders["val"],
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        phase="val",
        device=cfg.general.device,
    )

    print(f"\nval_rate_non_zero = {val_rate_non_zero}")

    print("\n=> Testing:\n")

    recall_at_n, recall_at_one_percent, mean_top1_distance = test(
        model=model,
        descriptor_key=cfg.general.test_modality,
        dataloader=dataloaders["test"],
        device=cfg.general.device,
    )

    stats_dict = {}
    stats_dict["test"] = {
        "mean_top1_distance": mean_top1_distance,
        "recall_at_1%": recall_at_one_percent,
        "recall_at_1": recall_at_n[0],
        "recall_at_3": recall_at_n[2],
        "recall_at_5": recall_at_n[4],
        "recall_at_10": recall_at_n[9],
    }
    stats_dict["train"] = train_stats
    stats_dict["train"]["batch_size"] = train_batch_size
    stats_dict["val"] = val_stats
    stats_dict["val"]["batch_size"] = val_batch_size

    # saving checkpoints
    checkpoint_dict = {
        "epoch": epoch + 1,
        "config": cfg,
        "stats_dict": stats_dict,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    torch.save(checkpoint_dict, checkpoints_dir / f"epoch_{epoch+1}.pth")
    # wandb logging
    if not cfg.general.debug and not cfg.wandb.disabled:
        wandb.log(flatten_dict(stats_dict))
        wandb.save(str((checkpoints_dir / f"epoch_{epoch+1}.pth").relative_to(".")))
    if recall_at_n[0] > best_recall_at_1:
        print("Recall@1 improved!")
        torch.save(checkpoint_dict, checkpoints_dir / "best.pth")
        best_recall_at_1 = recall_at_n[0]
        if not cfg.general.debug and not cfg.wandb.disabled:
            wandb.save(str((checkpoints_dir / "best.pth").relative_to(".")))



=====> Epoch 1:

=> Training:



train: 100%|██████████| 377/377 [00:03<00:00, 95.11it/s] 


train_rate_non_zero = 0.7397372742200329

=> Validating:




val: 100%|██████████| 18/18 [00:04<00:00,  4.26it/s]



val_rate_non_zero = 0.9881294173404789

=> Testing:



...


train_rate_non_zero = 0.8330644319131161

=> Validating:




val: 100%|██████████| 18/18 [00:04<00:00,  4.02it/s]



val_rate_non_zero = 0.9996461429582448

=> Testing:



Calculating metrics: 100%|██████████| 90/90 [00:54<00:00,  1.65it/s]

Mean Recall@N:
[0.22754518 0.34447911 0.41833438 0.47378735 0.51778284 0.55131168
 0.57875839 0.60390781 0.62472482 0.64419133 0.66160436 0.67753302
 0.69308212 0.70690534 0.71884605 0.72956926 0.73915765 0.7472358
 0.75627226 0.76370284 0.77140821 0.77828737 0.78548221 0.79179452
 0.79738323]
Mean Recall@1% = 0.5430542202799687
Mean top-1 distance = 0.5633553060844829
Recall@1 improved!


In [14]:
import torch.nn as nn

class TextModule(nn.Module):
    def __init__(self, text_emb_size=100, hidden_size=100):
        super().__init__()
        self.fc1 = nn.Linear(text_emb_size, hidden_size)
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, embedding):
        x = self.fc1(embedding)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [15]:
model = TextModule()

In [18]:
model(batch["text_emb_cam1"])

tensor([[-9.6135e-03, -1.0577e-01,  8.7988e-02, -2.3182e-02,  1.1544e-01,
          1.1407e-04, -6.5322e-02,  4.3350e-03,  2.6404e-03, -1.9650e-02,
         -4.0838e-02,  6.1159e-02, -1.6879e-01,  9.8357e-02, -4.2016e-03,
          2.9686e-03,  1.2739e-03, -7.9826e-02, -5.5356e-02, -9.6956e-02,
          3.2541e-02,  8.7272e-02,  6.0897e-02, -1.0772e-01,  7.4620e-02,
         -5.5626e-02,  8.2666e-02, -2.2510e-02,  9.8700e-02,  7.0989e-02,
         -1.1105e-01,  7.0539e-02,  6.2379e-02, -1.8779e-02,  2.2416e-02,
         -9.5671e-02, -8.8208e-03, -2.8558e-02, -7.1731e-02, -1.8757e-02,
         -4.6004e-02,  5.4501e-02,  6.0142e-03, -1.6162e-02, -6.1365e-03,
         -5.0851e-02, -1.0993e-01, -1.0689e-01,  2.7982e-02,  2.3153e-02,
         -1.7139e-02, -2.7945e-02,  3.5807e-02, -5.4931e-02, -9.9132e-02,
         -1.0072e-01, -7.1896e-02, -1.9962e-02, -4.1751e-02,  2.4051e-02,
          7.1095e-03, -5.3357e-02, -1.1899e-01, -8.3624e-03,  7.1089e-02,
          1.4980e-02,  4.2275e-02, -4.

In [20]:
t = torch.Tensor(1, 100)
t

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          3.1389e-43,  0.0000e+00,  1.4013e-45,  0.0000e+00,  9.1844e-41,
          1.1551e-40,  1.1481e-41,  2.0739e-43,  1.0712e+29,  0.0000e+00,
          4.4982e-43,  0.0000e+00,  7.7751e+28,  0.0000e+00,  1.1743e+29,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  4.5849e-41,
          6.7262e-44,  0.0000e+00,  2.9147e-43,  0.0000e+00,  2.4687e+15,
          4.5850e-41,  0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  7.7868e+28,  0.0000e+00,  2.0792e-36,  0.0000e+00,
         -7.8279e-28,  4.5849e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.

In [21]:
t.requires_grad = True

In [22]:
t

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.4013e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          3.1389e-43,  0.0000e+00,  1.4013e-45,  0.0000e+00,  9.1844e-41,
          1.1551e-40,  1.1481e-41,  2.0739e-43,  1.0712e+29,  0.0000e+00,
          4.4982e-43,  0.0000e+00,  7.7751e+28,  0.0000e+00,  1.1743e+29,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  0.0000e+00,  4.5849e-41,
          6.7262e-44,  0.0000e+00,  2.9147e-43,  0.0000e+00,  2.4687e+15,
          4.5850e-41,  0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  7.7868e+28,  0.0000e+00,  2.0792e-36,  0.0000e+00,
         -7.8279e-28,  4.5849e-41,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.